In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/THAThack_469', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(anon)

           id                date  latitude  longitude  week
3         172 2015-03-15 13:17:00  4.623548  46.008065    11
4         708 2015-04-15 04:19:00  5.146372  45.632107    16
5         669 2015-04-15 10:14:00  4.862442  45.777247    16
6         660 2015-03-27 10:17:00  4.873223  45.783843    13
7         426 2015-04-05 05:44:00  5.146367  45.631990    14
...       ...                 ...       ...        ...   ...
34551844  742 2015-04-15 18:03:00  4.873750  45.783215    16
34551845  854 2015-04-01 19:49:00  4.865855  45.755910    14
34551846  254 2015-04-05 12:10:00  4.872410  45.782332    14
34551847   84 2015-03-25 17:12:00  4.859868  45.781972    13
34551848  730 2015-04-26 05:31:00  4.869085  45.775523    17

[20129969 rows x 5 columns]


In [4]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['1' '1001' '1002' '1005' '1007' '1008' '1015' '1016' '1017' '102' '1022'
 '1023' '1026' '1029' '1034' '1039' '1041' '1047' '1050' '1053' '1054'
 '1057' '1059' '115' '116' '117' '118' '12' '123' '125' '128' '133' '138'
 '139' '140' '143' '144' '148' '15' '152' '154' '155' '157' '161' '163'
 '165' '172' '173' '175' '177' '18' '184' '19' '192' '193' '195' '199'
 '200' '202' '203' '205' '206' '209' '215' '217' '219' '22' '223' '224'
 '225' '227' '228' '231' '241' '25' '250' '254' '261' '262' '263' '271'
 '272' '273' '276' '278' '28' '286' '291' '294' '298' '299' '300' '301'
 '302' '304' '305' '306' '310' '314' '317' '318' '319' '32' '321' '322'
 '325' '326' '327' '328' '332' '333' '334' '335' '338' '34' '341' 

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [7]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(distance*delta_ratio*delta_hour*delta_time) #*delta_count*delta_time*distance*delta_hour
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

307
{'1': {'2015-10': ['84'], '2015-11': ['628'], '2015-12': ['448'], '2015-13': ['743'], '2015-14': ['584'], '2015-15': ['1026'], '2015-16': ['357'], '2015-17': ['639'], '2015-18': ['485'], '2015-19': ['102'], '2015-20': ['984']}, '2': {'2015-10': ['788'], '2015-11': ['985'], '2015-12': ['469'], '2015-13': ['883'], '2015-14': ['219'], '2015-15': ['678'], '2015-16': ['804'], '2015-17': ['535'], '2015-18': ['395'], '2015-19': ['28'], '2015-20': ['739']}, '107': {'2015-10': ['704'], '2015-11': ['263'], '2015-12': ['808']}, '98': {'2015-10': ['863'], '2015-11': ['231']}, '21': {'2015-10': ['688'], '2015-11': ['800']}, '5': {'2015-10': ['633'], '2015-11': ['305']}, '9': {'2015-10': ['720'], '2015-11': ['806'], '2015-12': ['314'], '2015-13': ['816'], '2015-14': ['514'], '2015-15': ['649'], '2015-16': ['390'], '2015-19': ['943']}, '6': {'2015-10': ['74'], '2015-11': ['971'], '2015-12': ['737'], '2015-13': ['74'], '2015-14': ['426'], '2015-15': ['694'], '2015-16': ['708']}, '7': {'2015-10': [

In [8]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "84"
        ],
        "2015-11": [
            "628"
        ],
        "2015-12": [
            "448"
        ],
        "2015-13": [
            "743"
        ],
        "2015-14": [
            "584"
        ],
        "2015-15": [
            "1026"
        ],
        "2015-16": [
            "357"
        ],
        "2015-17": [
            "639"
        ],
        "2015-18": [
            "485"
        ],
        "2015-19": [
            "102"
        ],
        "2015-20": [
            "984"
        ]
    },
    "2": {
        "2015-10": [
            "788"
        ],
        "2015-11": [
            "985"
        ],
        "2015-12": [
            "469"
        ],
        "2015-13": [
            "883"
        ],
        "2015-14": [
            "219"
        ],
        "2015-15": [
            "678"
        ],
        "2015-16": [
            "804"
        ],
        "2015-17": [
            "535"
        ],
        "2015-18": [